# Comparing various genotyping strategies

In [1]:
library(tidyverse)
library(here)

devtools::load_all(".")

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
here() starts at /mnt/expressions/mp/archaic-ychr
Loading ychr


# A00

We can clearly see that the strict consensus produces the smalles number of genotypes. The fact that the difference is fairly small is because the rate of sequencing errors is expected to be relatively low. However, the consensus-vs-tolerance difference is much more extreme for the ancient Y chromosomes below due to significant proportion of false aDNA damage alleles in many of the reads.

In [2]:
read_vcf(here("test/a00_baq.vcf.gz"), mindp = 3, maxdp = 0.98) %>% nrow

[1] 6909242

In [3]:
read_vcf(here("test/a00_nobaq.vcf.gz"), mindp = 3, maxdp = 0.98) %>% nrow

[1] 6909242

In [4]:
read_vcf(here("test/a00_consensus.vcf.gz"), mindp = 3, maxdp = 0.98) %>% nrow

[1] 6789749

In [5]:
read_vcf(here("test/a00_tolerance.vcf.gz"), mindp = 3, maxdp = 0.98) %>% nrow

[1] 6873078

In [6]:
a00 <- read_vcf(here("test/genotyping_a00.vcf.gz"), mindp = 3, maxdp = 0.98) %>%
    filter(!is.na(baq) | !is.na(nobaq) | !is.na(cons) | !is.na(tol))

In [7]:
nrow(a00)

[1] 6777849

Sites which are differently called under bcftools with or without BAQ option are very weird and manual inspection of the alignments shows that it's pretty much impossible to tell which are the true variants sites and which are errors. I don't really understand why does bcftools even call an allele in cases like this, but I guess they decided on slightly higher false positive rate and put the responsibility for filtering on the user.

I'd say being conservative in these particular cases is a good thing (note that my bam-caller decides to rather not call anything at these weird sites). The situation is even worse for the archaic samples below, where we often have a 50:50 mixture of two alleles and bcftools calls pretty much randomly one or another, which is not great.

In [8]:
filter(a00, baq != nobaq)

chrom,pos,REF,ALT,baq,nobaq,cons,tol
<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Y,2805552,G,A,0,1,NA,NA
Y,3715248,A,G,0,1,NA,NA
Y,8561368,C,T,0,1,NA,NA
Y,8561376,C,T,0,1,NA,NA
Y,17029481,G,A,0,1,NA,NA
Y,17029484,T,A,0,1,NA,NA


In [9]:
filter(a00, baq != nobaq) %>% group_by(baq, nobaq) %>% tally

baq,nobaq,n
<dbl>,<dbl>,<int>
0,1,6


In [10]:
filter(a00, is.na(cons) & !(is.na(cons) & is.na(tol))) %>%
    group_by(alts = ALT == "") %>% sample_n(10) %>% ungroup %>% select(-alts)

chrom,pos,REF,ALT,baq,nobaq,cons,tol
<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Y,8525740,T,C,1,1,NA,1
Y,9872017,T,C,1,1,NA,1
Y,18069115,A,G,1,1,NA,1
Y,7180949,G,A,1,1,NA,1
Y,19128936,A,C,NA,1,NA,1
Y,23052802,A,G,1,1,NA,1
Y,8124473,T,G,1,1,NA,1
Y,14076115,C,G,1,1,NA,1
Y,6868118,A,G,1,1,NA,1


In [11]:
filter(a00, tol != baq | tol != nobaq)

chrom,pos,REF,ALT,baq,nobaq,cons,tol
<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>


Based on the table right above, whenever we call an allele with the 90% tolerance consensus cutoff these calls are consistent with bcftools. So the only thing we are losing are calls which would be unreliable anyway, based on the manual inspection of alignments above.

# Mezmaiskaya 2 (high-coverage archaic, ideal case)

In [12]:
read_vcf(here("test/mez2_baq.vcf.gz"), mindp = 3, maxdp = 0.98) %>% nrow

[1] 6670383

In [13]:
read_vcf(here("test/mez2_nobaq.vcf.gz"), mindp = 3, maxdp = 0.98) %>% nrow

[1] 6670383

Notice the crazy drop in called SNPs when we do 100% strict consensus! This is the effect of aDNA damage SNPs throughout aDNA reads.

In [14]:
read_vcf(here("test/mez2_consensus.vcf.gz"), mindp = 3, maxdp = 0.98) %>% nrow

[1] 5502319

In [15]:
read_vcf(here("test/mez2_tolerance.vcf.gz"), mindp = 3, maxdp = 0.98) %>% nrow

[1] 6348948

In [16]:
mez2 <- read_vcf(here("test/genotyping_mez2.vcf.gz"), mindp = 3, maxdp = 0.98) %>%
    filter(!is.na(baq) | !is.na(nobaq) | !is.na(cons) | !is.na(tol))

In [17]:
nrow(mez2)

[1] 6359829

Again, sites where baq recalibration leads to different genotype calls than nobaq are weird and often look like heterozygotes (we correctly ignore them as suspicious):

In [18]:
filter(mez2, baq != nobaq) %>% sample_n(20)

chrom,pos,REF,ALT,baq,nobaq,cons,tol
<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Y,15363020,T,C,0,1,NA,NA
Y,19089753,T,C,0,1,NA,NA
Y,17572425,C,T,0,1,NA,NA
Y,18614096,A,G,0,1,NA,NA
Y,9984843,G,A,0,1,NA,NA
Y,15516284,C,T,0,1,NA,NA
Y,18038445,C,T,0,1,NA,NA
Y,22160922,C,T,1,0,NA,NA
Y,7280606,T,C,0,1,NA,NA


In [19]:
filter(mez2, baq != nobaq) %>% group_by(baq, nobaq) %>% tally

baq,nobaq,n
<dbl>,<dbl>,<int>
0,1,35
1,0,5


There shouldn't be any difference between strict and less strict consensus at sites called by both:

In [20]:
filter(mez2, cons != tol)

chrom,pos,REF,ALT,baq,nobaq,cons,tol
<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>


In [21]:
filter(mez2, is.na(cons) & !(is.na(cons) & is.na(tol))) %>%
    group_by(alts = ALT == "") %>% sample_n(10) %>% ungroup %>% select(-alts)

chrom,pos,REF,ALT,baq,nobaq,cons,tol
<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Y,18596979,A,G,1,1,NA,1
Y,20837722,A,G,1,1,NA,1
Y,19141658,T,C,1,1,NA,1
Y,21460975,G,A,1,1,NA,1
Y,18060417,C,G,1,1,NA,1
Y,19481868,A,C,1,1,NA,1
Y,8193347,C,G,1,1,NA,1
Y,18113973,A,G,1,1,NA,1
Y,17594650,T,C,1,1,NA,1


The looser 90% tolerance consensus matches perfectly the calls made by bcftools.

In [22]:
filter(mez2, tol != baq | tol != nobaq)

chrom,pos,REF,ALT,baq,nobaq,cons,tol
<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>


# Denisova 8 (low coverage archaic, extreme case)

At 90% consensus cutoff we expect the lower coverage genotypes to be very similar to strict 100% consensus (unlike high-coverage Mezmaiskaya 2).

In [23]:
read_vcf(here("test/den8_baq.vcf.gz"), mindp = 3, maxdp = 0.98) %>% nrow

[1] 6261470

In [24]:
read_vcf(here("test/den8_nobaq.vcf.gz"), mindp = 3, maxdp = 0.98) %>% nrow

[1] 6261470

In [25]:
read_vcf(here("test/den8_consensus.vcf.gz"), mindp = 3, maxdp = 0.98) %>% nrow

[1] 5831935

In [26]:
read_vcf(here("test/den8_tolerance.vcf.gz"), mindp = 3, maxdp = 0.98) %>% nrow

[1] 5851332

In [27]:
den8 <- read_vcf(here("test/genotyping_den8.vcf.gz"), mindp = 3, maxdp = 0.98) %>%
    filter(!is.na(baq) | !is.na(nobaq) | !is.na(cons) | !is.na(tol))

In [28]:
nrow(den8)

[1] 4185738

In [29]:
filter(den8, baq != nobaq) %>% sample_n(20)

chrom,pos,REF,ALT,baq,nobaq,cons,tol
<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Y,15002613,C,G,0,1,NA,NA
Y,3717723,C,T,0,1,NA,NA
Y,16340614,C,T,0,1,NA,NA
Y,14115977,C,T,0,1,NA,NA
Y,17097106,C,T,0,1,NA,NA
Y,18822816,C,T,0,1,NA,NA
Y,8735945,G,A,0,1,NA,NA
Y,22702086,C,G,0,1,NA,NA
Y,15435772,G,A,0,1,NA,NA


In [30]:
filter(den8, baq != nobaq) %>% group_by(baq, nobaq) %>% tally

baq,nobaq,n
<dbl>,<dbl>,<int>
0,1,224
1,0,11


In [31]:
filter(den8, cons != tol)

chrom,pos,REF,ALT,baq,nobaq,cons,tol
<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>


In [32]:
filter(den8, is.na(cons) & !(is.na(cons) & is.na(tol))) %>%
    group_by(alts = ALT == "") %>% sample_n(3) %>% ungroup %>% select(-alts)

chrom,pos,REF,ALT,baq,nobaq,cons,tol
<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Y,8396636,A,G,1,1,NA,1
Y,17517362,A,G,1,1,NA,1
Y,18759298,A,G,1,1,NA,1
Y,15941841,C,,0,0,NA,0
Y,2710399,G,,0,0,NA,0
Y,22550838,G,,0,0,NA,0


In [33]:
filter(den8, tol != baq | tol != nobaq)

chrom,pos,REF,ALT,baq,nobaq,cons,tol
<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>


# Spy 1 (low coverage archaic, even more extreme case)

At 90% consensus cutoff we expect the lower coverage genotypes to be the same as strict 100% consensus, same as for Denisova 8 analysed above.

In [34]:
read_vcf(here("test/spy1_baq.vcf.gz"), mindp = 3, maxdp = 0.98) %>% nrow

[1] 3558957

In [35]:
read_vcf(here("test/spy1_nobaq.vcf.gz"), mindp = 3, maxdp = 0.98) %>% nrow

[1] 3558957

In [36]:
read_vcf(here("test/spy1_consensus.vcf.gz"), mindp = 3, maxdp = 0.98) %>% nrow

[1] 3502149

In [37]:
read_vcf(here("test/spy1_tolerance.vcf.gz"), mindp = 3, maxdp = 0.98) %>% nrow

[1] 3502380

In [38]:
spy1 <- read_vcf(here("test/genotyping_spy1.vcf.gz"), mindp = 3, maxdp = 0.98) %>%
    filter(!is.na(baq) | !is.na(nobaq) | !is.na(cons) | !is.na(tol))

In [39]:
nrow(spy1)

[1] 611817

In [40]:
filter(spy1, baq != nobaq) %>% sample_n(20)

chrom,pos,REF,ALT,baq,nobaq,cons,tol
<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Y,21843326,G,A,1,0,NA,NA
Y,15833781,G,A,1,0,NA,NA
Y,6792204,G,A,0,1,NA,NA
Y,8458374,C,T,0,1,NA,NA
Y,8707564,G,A,0,1,NA,NA
Y,23030051,C,T,0,1,NA,NA
Y,7198293,C,A,0,1,NA,NA
Y,19341925,G,A,0,1,NA,NA
Y,15315939,G,A,0,1,NA,NA


In [41]:
filter(spy1, baq != nobaq) %>% group_by(baq, nobaq) %>% tally

baq,nobaq,n
<dbl>,<dbl>,<int>
0,1,61
1,0,14


In [42]:
filter(spy1, cons != tol)

chrom,pos,REF,ALT,baq,nobaq,cons,tol
<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>


In [43]:
filter(spy1, is.na(cons) & !(is.na(cons) & is.na(tol))) %>%
    group_by(alts = ALT == "") %>% sample_n(10) %>% ungroup %>% select(-alts)

chrom,pos,REF,ALT,baq,nobaq,cons,tol
<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>


In [44]:
filter(spy1, tol != baq | tol != nobaq)

chrom,pos,REF,ALT,baq,nobaq,cons,tol
<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
